<a href="https://colab.research.google.com/github/zum-m/jouhougakugairon/blob/main/VGG16_frozen_kadai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.datasets import cifar10
import numpy as np

(x_train, y_train),(x_test, y_test) = cifar10.load_data()
#データの前処理
x_train=preprocess_input(x_train)
x_test=preprocess_input(x_test)

#正解ラベルをOne-Hot表現に変換
y_train_onehot = np.eye(10)[y_train]
y_train_onehot = np.squeeze(y_train_onehot)
y_test_onehot = np.eye(10)[y_test]
y_test_onehot = np.squeeze(y_test_onehot)

170498071/170498071 [==============================] - 5s 0us/step


In [5]:
#学習済みのVGG16を特徴抽出器として使う
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

#モデルの設計
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(10,activation='softmax'))

model.summary()
#モデルの学習

# VGG16の重みを凍結して更新されないようにする
conv_base.trainable = False

conv_base.summary()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 activation_1 (Activation)   (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 10)               

In [6]:
early_stopping = EarlyStopping(monitor='loss', min_delta=0.01, patience=2)

#learning_rate は低めに設定
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])
history = model.fit(x_train, y_train_onehot, batch_size=200, epochs=100, callbacks=[early_stopping])

#モデルの評価
loss, acc = model.evaluate(x_test,y_test_onehot)
print('Test loss:', loss, 'Test accuracy:', acc)

Epoch 1/100
250/250 [==============================] - 15s 22ms/step - loss: 8.7913 - accuracy: 0.1590
Epoch 2/100
250/250 [==============================] - 5s 22ms/step - loss: 4.6547 - accuracy: 0.2694
Epoch 3/100
250/250 [==============================] - 6s 23ms/step - loss: 3.5467 - accuracy: 0.3282
Epoch 4/100
250/250 [==============================] - 5s 22ms/step - loss: 2.9209 - accuracy: 0.3610
Epoch 5/100
250/250 [==============================] - 6s 22ms/step - loss: 2.5130 - accuracy: 0.3834
Epoch 6/100
250/250 [==============================] - 5s 22ms/step - loss: 2.2267 - accuracy: 0.4008
Epoch 7/100
250/250 [==============================] - 5s 22ms/step - loss: 2.0233 - accuracy: 0.4169
Epoch 8/100
250/250 [==============================] - 6s 25ms/step - loss: 1.8777 - accuracy: 0.4321
Epoch 9/100
250/250 [==============================] - 6s 22ms/step - loss: 1.7712 - accuracy: 0.4477
Epoch 10/100
250/250 [==============================] - 6s 23ms/step - loss: 1.68